In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import math 
from sympy import * 


In [40]:
from tensorflow import keras 

In [2]:
'''' 
RSI - (Relative Strength Index) - Technical Indicator in Trading used to find if the trend observed in previous days 
will continue or NOT . 


[RSI = 100 - 100/1+RS ] where RS (relative Strength_ ) = Avg Gain/ Avg Loss over past some days (g/L calcualted wrt prev Days CLosing Price )

if ( 0 < RSI < 30 ) , oversold and price was decreasing till NOW  and now increase so buy it (LONG POSITIOn)
and vice versa for ( 70 < RSI < 100 ) -> SHORT (sell it)

'''

"' \nRSI - (Relative Strength Index) - Technical Indicator in Trading used to find if the trend observed in previous days \nwill continue or NOT . \n\n\n[RSI = 100 - 100/1+RS ] where RS (relative Strength_ ) = Avg Gain/ Avg Loss over past some days (g/L calcualted wrt prev Days CLosing Price )\n\nif ( 0 < RSI < 30 ) , oversold and price was decreasing till NOW  and now increase so buy it (LONG POSITIOn)\nand vice versa for ( 70 < RSI < 100 ) -> SHORT (sell it)\n\n"

In [3]:
# COde the RSI 

import yfinance as yf  # for historical stock martket data 

data = yf.download('SPY')

[*********************100%%**********************]  1 of 1 completed


In [4]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-29,43.96875,43.96875,43.75000,43.93750,24.684090,1003200
1993-02-01,43.96875,44.25000,43.96875,44.25000,24.859673,480500
1993-02-02,44.21875,44.37500,44.12500,44.34375,24.912336,201300
1993-02-03,44.40625,44.84375,44.37500,44.81250,25.175680,529400
1993-02-04,44.96875,45.09375,44.46875,45.00000,25.281029,531500


In [5]:
import pandas_ta as ta 

In [6]:
'''  

Cl;osing Price (CLose) - The price at which the last stock is traded in the day . 

Adjusted CLosing Price  - It is the last price at which a stock , dividends and all other type of assets are traded in the same day. 

Adj Closing Price - keep s happenng after close 
and is more clear indicator of stock market's  health . 

'''

"  \n\nCl;osing Price (CLose) - The price at which the last stock is traded in the day . \n\nAdjusted CLosing Price  - It is the last price at which a stock , dividends and all other type of assets are traded in the same day. \n\nAdj Closing Price - keep s happenng after close \nand is more clear indicator of stock market's  health . \n\n"

In [7]:
# import pandas_ta as ta 

# data['rsi'] = ta.rsi(data['Adj Close'] , 2 )

In [8]:
''' # function to calculate RSI Prices (for each day ) 

# df.diff() -> return difference b/w all the rows and its just previous row (if period==1 )
# .ewm(com ) -> finds exponential moving average ( weighted sum of the data by giving more weight to more recent obs. abnd less weight toolder observ.)


'''

' # function to calculate RSI Prices (for each day ) \n\n# df.diff() -> return difference b/w all the rows and its just previous row (if period==1 )\n# .ewm(com ) -> finds exponential moving average ( weighted sum of the data by giving more weight to more recent obs. abnd less weight toolder observ.)\n\n\n'

In [9]:

# work for every row (dat) of today's close price

def calculate_rsi(prices , period) :

    momentum = prices.diff() # shows the change in price over a day wrt prev day's adj close price 
    gain = abs (momentum.where (momentum  > 0 , 0 ) )
    loss = abs ( momentum.where(momentum < 0 , 0 ) )
    
    avg_gain = gain.ewm(com=period - 1, min_periods=period).mean()  # exponential movbving AVERAGE 
    avg_loss = loss.ewm(com=period - 1, min_periods=period).mean() 

    rs = avg_gain/ avg_loss

    rsi = 100 - (100 / (1 + rs ))

    return rsi 

In [10]:
def generate_signals(rsi_list) :

    signals = [] 

    for rsi in rsi_list :

        if rsi < 30 :
            signals.append('BUY') 
        elif rsi > 70 :
            signals.append('SELL')
        else :
            signals.append('HOLD')
        
    return signals 

In [11]:
rsi_list = calculate_rsi(data['Adj Close'] , 14 )

In [12]:
data['RSI'] = rsi_list

In [13]:
data['Signals'] = generate_signals(rsi_list)

In [14]:
data['Signals'].value_counts()

Signals
HOLD    7133
SELL     662
BUY      112
Name: count, dtype: int64

In [15]:
# Newton Raphson's Method of Apptoximation 
# New approx ->   x(n+1)  = x(n) - f(xn)/f'(xn) 



# for complex functions root -> slike sin(x_ + cos3(x) )
def Newton_Raphon_Approx( eqn,  initial_guess ) :
    x_n = initial_guess 

    x = symbols('x')
    eqn_d_x = Derivative( eqn , x).doit() # to simplify the expression  

    x_n_1 = x_n - eqn.subs(x , x_n)/eqn_d_x.subs(x , x_n) # eqn.subs - sunstitutes  x in x_n

    while abs(x_n_1 - x_n) >= 0.0000004 : 
        temp = x_n_1 
        x_n_1 = x_n - eqn.subs(x , x_n)/eqn_d_x.subs(x , x_n)
        x_n =temp 

    return x_n_1 



x = symbols('x')

# USe Sympy to write compelx expression / eqn 
res = Newton_Raphon_Approx( x**2 -1  ,  1.0001)


print( res , " = " , res.evalf())

1.00000000499950  =  1.00000000499950
